In [260]:
# imports
import pandas as pandas
import scipy.stats as stats
import numpy as np

import copy
import random

from Bio import SeqIO
from Bio.KEGG.REST import *
from Bio.KEGG.KGML import KGML_parser
from Bio.Graphics.KGML_vis import KGMLCanvas
from Bio.Graphics.ColorSpiral import ColorSpiral
from IPython.display import Image, HTML

In [5]:
# copied from 

# A bit of code that will help us display the PDF output
def PDF(filename):
    return HTML('<iframe src=%s width=700 height=350></iframe>' % filename)

# A bit of helper code to shorten long text
def head(text, lines=10):
    """ Print the first lines lines of the passed text.
    """
    print('\n'.join(text.split('\n')[:lines] + ['[...]']))

## Task 1: KEGG and gene id mapping

Familiarize yourself with the KEGG Rest interface and how to access it with Biopyhton:

http://www.genome.jp/kegg/rest/keggapi.html

http://nbviewer.jupyter.org/github/widdowquinn/notebooks/blob/master/Biopython_KGML_intro.ipynb

### Subtask 1.1 Extract gene lists for all (mouse) KEGG pathways and store them in a suitable Python data structure

In [123]:
# download mus musculus pathways
mus_musculus_pathways = kegg_list("pathway", "mmu").read()

# convert into DataFrame
# pathway line format:
# path:<id>\t<description>\n
df = pandas.DataFrame([line.replace(':', '\t', 1).split('\t') for line in mus_musculus_pathways.split('\n')], columns = ['Org', 'ID', 'Description'])

# remove last row containing 'NaN'
df = df[:-1]

# set index
df.set_index('ID', inplace=True)



In [124]:
df.head()

,Org,Description
ID,,
mmu00010,path,Glycolysis / Gluconeogenesis - Mus musculus (m...
mmu00020,path,Citrate cycle (TCA cycle) - Mus musculus (mouse)
mmu00030,path,Pentose phosphate pathway - Mus musculus (mouse)
mmu00040,path,Pentose and glucuronate interconversions - Mus...
mmu00051,path,Fructose and mannose metabolism - Mus musculus...


In [189]:
# get genes of its corresponding pathway

def getGenes(entry):
    """
    Parses a KEGG database entry and returns all genes in this entry
    as a list.
    """
    
    # gene list is parsed or not
    readGenes = False
    
    # list of genes correspond to pathway
    genes = []

    for line in entry.split('\n'):
        if line.startswith('GENE'):
            readGenes=True
        if line.startswith('COMPOUND'):
            # all genes parsed
            readGenes=False
        if readGenes:
            '''
            Every line in GENES has the format
            
            <id number>    <gene symbol>;  <anything else>
            
            For the first line the identifier GENE has to be removed. Then for every line
            remove the ';' delimiter, split the line at whitespaces and, if the length of
            the resulting list is > 2 (i.e. contains genes), take the second entry (gene symbol).
            '''
            
            # while gene entry is parsed, split entries and get gene id
            entries_list = line.replace('GENE', '').replace(';', '').split()
            if len(entries_list) > 2:
                genes.append(entries_list[1])
    
    return genes

In [236]:
# get dict of all genes to its corresponding pathway
gene_set = {pathway:getGenes(kegg_get(pathway).read()) for pathway in df.index}

# gene_set is now a dictionary with following entries:
# {'mmu00010': ['Hk2', 'Hk3', 'Hk1', 'Hkdc1', 'Gck', ...], 'mmu00020':[...], ...}

KeyboardInterrupt: 

### Subtask 1.2: Save the KEGG gene sets as a gmt file after you made sure they have the proper gene ids with respect to your DE analysis

hints: 

http://www.broadinstitute.org/cancer/software/gsea/wiki/index.php/Data_formats

In [261]:
# safe copy of the gene_set to avoid re-creating gene list during debugging
genes_copy = copy.deepcopy(gene_set)

# add Org and Description entry to dictionary
for entry in genes_copy:
    genes_copy[entry].insert(0, df.loc[entry]['Org'])
    genes_copy[entry].insert(1, df.loc[entry]['Description'])

# genes_copy is now a dictionary with following entries:
# {'mmu00010': ['<Org>', '<description>', Hk2', 'Hk3', 'Hk1', 'Hkdc1', 'Gck', ...], 'mmu00020':[...], ...}

# safe dictionary as DataFrame to make export to GMT easier
full_df = pandas.DataFrame.from_dict(genes_copy, orient='index')

# export as gmt
full_df.to_csv('pathwayOutput/KEGG_genes.gmt')
# as well as csv to verify correct output format using Excel
full_df.to_csv('pathwayOutput/KEGG_genes.csv')

full_df.head()

,0,1,2,3,4,5,6,7,8,9,...,1114,1115,1116,1117,1118,1119,1120,1121,1122,1123
mmu00010,path,Glycolysis / Gluconeogenesis - Mus musculus (m...,Hk2,Hk3,Hk1,Hkdc1,Gck,Gpi1,Pfkl,Pfkm,...,None,None,None,None,None,None,None,None,None,None
mmu00020,path,Citrate cycle (TCA cycle) - Mus musculus (mouse),Cs,Csl,Acly,Aco2,Aco1,Idh1,Idh2,Idh3g,...,None,None,None,None,None,None,None,None,None,None
mmu00030,path,Pentose phosphate pathway - Mus musculus (mouse),Gpi1,G6pd2,G6pdx,Pgls,H6pd,Pgd,Rpe,Tkt,...,None,None,None,None,None,None,None,None,None,None
mmu00040,path,Pentose and glucuronate interconversions - Mus...,Gusb,Kl,Ugt2b5,Ugt1a2,Ugt1a6a,Ugt2a1,Ugt1a9,Ugt1a10,...,None,None,None,None,None,None,None,None,None,None
mmu00051,path,Fructose and mannose metabolism - Mus musculus...,Mpi,Pmm2,Pmm1,Gmppb,Gmppa,Gmds,Tsta3,Fpgt,...,None,None,None,None,None,None,None,None,None,None


## Task 2: Gene Set Enrichment

### Subtask 2.1: Read in the csv file you produced during the Differential Expression module, extract a gene list (as a python list of gene symbols) from your favorite multiple correction column (and store it in a variable)

In [259]:
mc_file = pandas.read_csv('outputFiles/MultipleCorrectionWithFoldChange.csv')
mc_file = mc_file.set_index('Gene')

diff_expr_genes_simes_hochberg = list(mc_file['simes-hochberg'])

mc_file.tail()

,mannwhitneyu pvalue,sidak,holm-sidak,holm,simes-hochberg,hommel,fdr_bh,fdr_by,fdr_tsbh,fdr_tsbky,l2fc
Gene,,,,,,,,,,,
Zzz3,0.000002,0.034391,0.029761,0.030213,0.030213,0.023989,0.000012,0.000124,0.000005,0.000005,0.142215
a,0.077554,1.000000,1.000000,1.000000,0.500000,0.500000,0.116936,1.000000,0.051345,0.054565,-0.074572
l7Rn6,0.014377,1.000000,1.000000,1.000000,0.500000,0.500000,0.028579,0.302032,0.012548,0.013335,-0.111329
mCG_21548,0.152689,1.000000,1.000000,1.000000,0.500000,0.500000,0.201806,1.000000,0.088610,0.094167,-0.043389
rp9,0.021855,1.000000,1.000000,1.000000,0.500000,0.500000,0.040614,0.429228,0.017833,0.018951,0.040655


### Subtask 2.2: Perform gene set enrichment (Fisher's exact test or an hypergeometric test will do for our purposes) with the KEGG gene sets you extracted in Task 1 (you may want to store the results in a pandas dataframe and write them to csv)

hint:

https://genetrail2.bioinf.uni-sb.de/help?topic=set_level_statistics

In [301]:
# get set of unique genes from kegg
l = set()
for idx in full_df.index:
    for itm in full_df.loc[idx][2:].values:
        l.add(itm)
        


{'Antinore', 'Spdye4b', 'Serpinb9d', 'Olfr605', 'P2rx5', 'Tdg-ps', 'Tbc1d4', 'Olfr728', 'Olfr357', 'Npffr1', 'Tnfrsf10b', 'Ppp1r1a', 'Ndufb7', 'Lasi', 'Sorbs1', 'Spta1', 'Crls1', 'Olfr1322', 'Fkbp4', 'Arrb1', 'Olfr1109', 'Rpl37a', 'Kcnmb3', 'Mgll', 'Galnt14', 'Pluripotent', 'Atp5h', 'Olfr616', 'Pfkp', 'Impact', 'Olfr129', 'Ndufa2', 'Olfr878', 'Zheng', 'Abca7', 'Nfatc3', 'Sugt1', 'Peg12', 'Ouyang', 'Abraxas1', 'Hmgcs2', 'Naip5', 'Cers2', 'Olfr1388', 'Strober', 'Olfr1298', 'Hinz', 'Etnppl', 'Naip7', 'Hspd1', 'Npy6r', 'Olfr1200', 'Best2', 'Nkx3-1', 'Olfr324', 'Mtr', 'Wimberly', 'Avpr2', 'Snca', 'Olfr262', 'Nakatogawa', 'Nceh1', 'Npc1', 'Yars', 'Nfasc', 'Flnc', 'Ndufa3', 'Olfr73', 'Cyp4a12b', 'Serpina1a', 'Nras', 'Adcy2', 'When', 'Abcg4', 'Hist1h2bg', 'Ppp2r5d', 'Tlr7', 'Ggt7', 'Biochim', 'Gm7030', 'Dio2', 'MYCN', 'Plcd1', 'Kumar-Sinha', 'Cdc42', 'Fabp2', 'Olfr541', 'Rhoa', 'Actb', 'Olfr214', 'U2af1l4', 'Mir7-2', 'Epo', 'Setd1b', 'Nt5c', 'Snap29', 'Csnk1g1', 'Olfr273', 'Mlh3', 'Stat5b', 'K

### Subtask 2.3: Extract a list of significantly (at 0.05 significance) enriched KEGG pathways

## Task 3: KEGG map visualization

#### hint:

http://nbviewer.jupyter.org/github/widdowquinn/notebooks/blob/master/Biopython_KGML_intro.ipynb

#### remark:

In real life you may want to use the R-based tool pathview: https://bioconductor.org/packages/release/bioc/html/pathview.html (if you insist you can also try to use r2py for using pathview from Python during the practical)

For Python (in addition to the Biopyhton module) https://github.com/idekerlab/py2cytoscape in combination with https://github.com/idekerlab/KEGGscape may be another alternative (in the future)

Generally speaking, it is always a good idea to pay attention also to other pathway databases like Reactome or WikiPathways ...

### Subtask 3.1: Pick some significantly enriched KEGG pathways of your choice from 2.3 and visualize them

### Subtask 3.2: Define a a suitable binary color scheme respresenting the fact whether a gene is significantly expressed or not

hint: 

http://www.rapidtables.com/web/color/RGB_Color.htm

### Subtask 3.3: Visualize the pathway(s) from 3.1 in such a way that the included genes have the corresponding color from 3.2 ( you may need to define a suitable mapping from single genes to what is actually shown in the pathway map...)